# Test model for classification

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns
import scipy
from scipy.stats import pearsonr
import sklearn
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import joblib

In [2]:
# Read both train dataset and test dataset
tf=pd.read_excel("TestDatasetExample.xls") # Change the file name here.
df=pd.read_excel("TrainDataset2023.xls")
tf.head() 
tf.describe()

,Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,TumourStage,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.0,...,3.000000,3.000000,3.000000,3.000000,3.000000e+00,3.000000,3.000000,3.000000,3.000000,3.000000
mean,58.371891,0.333333,0.333333,0.333333,0.666667,2.333333,2.333333,333.666667,0.333333,3.0,...,0.277656,0.277655,3.202574,0.003907,2.649985e+06,126.097192,0.004985,0.089091,0.009130,0.004266
std,1.563767,0.577350,0.577350,0.577350,0.577350,0.577350,1.154701,576.195569,0.577350,1.0,...,0.072521,0.072520,0.509325,0.003051,2.162805e+06,99.748099,0.005479,0.029038,0.005321,0.004599
min,56.881588,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,0.000000,2.0,...,0.194591,0.194591,2.846439,0.001281,1.736585e+05,23.967478,0.001334,0.055890,0.003163,0.001194
25%,57.557837,0.000000,0.000000,0.000000,0.500000,2.000000,2.000000,1.000000,0.000000,2.5,...,0.252295,0.252294,2.910878,0.002233,1.890740e+06,77.506009,0.001834,0.078759,0.007003,0.001622
50%,58.234086,0.000000,0.000000,0.000000,1.000000,2.000000,3.000000,1.000000,0.000000,3.0,...,0.309999,0.309996,2.975317,0.003185,3.607821e+06,131.044541,0.002335,0.101628,0.010844,0.002051
75%,59.117043,0.500000,0.500000,0.500000,1.000000,2.500000,3.000000,500.000000,0.500000,3.5,...,0.319188,0.319187,3.380641,0.005219,3.888148e+06,177.162048,0.006810,0.105691,0.012113,0.005802
max,60.000000,1.000000,1.000000,1.000000,1.000000,3.000000,3.000000,999.000000,1.000000,4.0,...,0.328377,0.328377,3.785966,0.007253,4.168474e+06,223.279556,0.011285,0.109755,0.013383,0.009553


In [3]:
# save IDs for output 
IDs = tf['ID']
tf.drop('ID', axis=1, inplace=True) # drop ID for prediction

In [4]:
# Check if there exists any missing value
tf.replace(999, np.nan, inplace=True)
for col in tf.columns:
    if tf[col].isnull().any():
        print(col, 'has missing values')
        if col == 'PgR' or 'HER2' or 'LNStatus' or 'TrippleNegative' or 'ChemoGrade': # using mode of train dataset to fill 
            tf[col] = tf[col].fillna(df[col].mode()[0])
        else:
            tf[col] = tf[col].fillna(df[col].median()) # using median of train dataset
tf.describe()

HistologyType has missing values


,Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,TumourStage,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.0,3.000000,3.0,...,3.000000,3.000000,3.000000,3.000000,3.000000e+00,3.000000,3.000000,3.000000,3.000000,3.000000
mean,58.371891,0.333333,0.333333,0.333333,0.666667,2.333333,2.333333,1.0,0.333333,3.0,...,0.277656,0.277655,3.202574,0.003907,2.649985e+06,126.097192,0.004985,0.089091,0.009130,0.004266
std,1.563767,0.577350,0.577350,0.577350,0.577350,0.577350,1.154701,0.0,0.577350,1.0,...,0.072521,0.072520,0.509325,0.003051,2.162805e+06,99.748099,0.005479,0.029038,0.005321,0.004599
min,56.881588,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.0,0.000000,2.0,...,0.194591,0.194591,2.846439,0.001281,1.736585e+05,23.967478,0.001334,0.055890,0.003163,0.001194
25%,57.557837,0.000000,0.000000,0.000000,0.500000,2.000000,2.000000,1.0,0.000000,2.5,...,0.252295,0.252294,2.910878,0.002233,1.890740e+06,77.506009,0.001834,0.078759,0.007003,0.001622
50%,58.234086,0.000000,0.000000,0.000000,1.000000,2.000000,3.000000,1.0,0.000000,3.0,...,0.309999,0.309996,2.975317,0.003185,3.607821e+06,131.044541,0.002335,0.101628,0.010844,0.002051
75%,59.117043,0.500000,0.500000,0.500000,1.000000,2.500000,3.000000,1.0,0.500000,3.5,...,0.319188,0.319187,3.380641,0.005219,3.888148e+06,177.162048,0.006810,0.105691,0.012113,0.005802
max,60.000000,1.000000,1.000000,1.000000,1.000000,3.000000,3.000000,1.0,1.000000,4.0,...,0.328377,0.328377,3.785966,0.007253,4.168474e+06,223.279556,0.011285,0.109755,0.013383,0.009553


In [5]:
# load power transformer
pt_loaded = joblib.load('PCR_power_transformer.pkl')
tf_transformed = pt_loaded.transform(tf)

In [6]:
# load standard scaler
scaler_loaded = joblib.load('PCR_standard_scaler.pkl')
scaler_transformed = scaler_loaded.transform(tf_transformed)

In [7]:
import pandas as pd

# Load the feature importances
importance_df = pd.read_csv('PCR_feature_importances.csv')
importances = importance_df['Importance'].values
feature_names = importance_df['Feature'].values

# Apply the same threshold the same as train data
threshold = np.sort(importances)[-100]

# Load the PCA model from the file
pca = joblib.load('PCR_pca.pkl')

In [8]:
# Apply feature selection the same as train data
Xs_test_rf = scaler_transformed[:, importances >= threshold]
X_pca_test = pca.transform(Xs_test_rf)

In [9]:
# load random forest regressor
import joblib
mlp_classifier_loaded = joblib.load('PCR_mlp_model.pkl')

In [10]:
# Predict
predictions = mlp_classifier_loaded.predict(X_pca_test)
print(predictions)

# Convert result to dataframe
predictions_df = pd.DataFrame(predictions, columns=['pCR'])
# concat id and results
result_df = pd.concat([IDs, predictions_df], axis=1)

# Save results to csv file
result_df.to_csv('OUTPUT_PCR_predictions.csv', index=False)

[1 1 0]
